<a href="https://colab.research.google.com/github/vardhanreddy2003/DeepLearning_projects/blob/main/plant_class_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
100% 2.04G/2.04G [01:43<00:00, 21.0MB/s]
100% 2.04G/2.04G [01:43<00:00, 21.2MB/s]


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile("/content/plantvillage-dataset.zip")
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [ ]:
train_ds=keras.utils.image_dataset_from_directory(
    directory="/content/plantvillage dataset/color",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256),
    validation_split=0.2,
    subset='training',
    seed=123
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory="/content/plantvillage dataset/color",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256),
    validation_split=0.2,
    subset='validation',
    seed=123
)

Found 54305 files belonging to 38 classes.
Using 43444 files for training.
Found 54305 files belonging to 38 classes.
Using 10861 files for validation.


In [ ]:
train_segmented_ds=keras.utils.image_dataset_from_directory(
    directory="/content/plantvillage dataset/color",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256),
    validation_split=0.2,
    subset='training',
    seed=123
)

validation_segemented_ds = keras.utils.image_dataset_from_directory(
    directory="/content/plantvillage dataset/color",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256),
    validation_split=0.2,
    subset='validation',
    seed=123
)

Found 54305 files belonging to 38 classes.
Using 43444 files for training.
Found 54305 files belonging to 38 classes.
Using 10861 files for validation.


In [ ]:


train_ds = train_ds.concatenate(train_segmented_ds)
validation_ds = validation_ds.concatenate(validation_segemented_ds)


In [ ]:


validation_ds_len = len(validation_ds)
test_ds_len = int(validation_ds_len * 0.2)
validation_ds_len = validation_ds_len - test_ds_len

test_ds = validation_ds.take(test_ds_len)
validation_ds = validation_ds.skip(test_ds_len)


In [ ]:
def preprocess(image, label):
  image = tf.cast(image, tf.float32) / 255.0  # Rescale to [0, 1]
  label = tf.one_hot(label, depth=38)
  return image, label

train_ds = train_ds.map(preprocess)
test_ds=test_ds.map(preprocess)
validation_ds = validation_ds.map(preprocess)


In [ ]:
model =Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(38, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 246016)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      62,980,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 38)                  │           9,766 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 63,009,510 (240.36 MB)

 Trainable params: 63,009,510 (240.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
history=model.fit(train_ds,batch_size=32,epochs=10,validation_data=test_ds)

Epoch 1/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 153s 56ms/step - accuracy: 0.9950 - loss: 0.0221 - val_accuracy: 0.8261 - val_loss: 2.1203
Epoch 2/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 200s 56ms/step - accuracy: 0.9954 - loss: 0.0204 - val_accuracy: 0.8481 - val_loss: 1.7757
Epoch 3/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 203s 56ms/step - accuracy: 0.9964 - loss: 0.0154 - val_accuracy: 0.8362 - val_loss: 2.4469
Epoch 4/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 154s 57ms/step - accuracy: 0.9957 - loss: 0.0213 - val_accuracy: 0.8470 - val_loss: 2.4683
Epoch 5/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 199s 56ms/step - accuracy: 0.9939 - loss: 0.0358 - val_accuracy: 0.8403 - val_loss: 2.3381
Epoch 6/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 153s 56ms/step - accuracy: 0.9970 - loss: 0.0167 - val_accuracy: 0.8502 - val_loss: 2.1988
Epoch 7/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 152s 56ms/step - accuracy: 0.9969 - loss: 0.0206 - val_accuracy: 0.8378 - val_loss: 2.3897
Epoch 8/10
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 152s 56ms/step - accuracy: 

In [ ]:
#model evaluation

loss, accuracy = model.evaluate(validation_ds)
print('Validation accuracy:', accuracy)


544/544 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.8482 - loss: 2.8741
Validation accuracy: 0.8466897010803223
